In [9]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Veri seti dizinleri
data_dir = "veri_yolu"  # Verilerin olduğu klasörü belirtin
categories = ["A", "B", "C"]  # Kullanacağınız etiketler
img_size = 300  # Görüntü boyutu

# Dummy veri seti oluştur
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Ensure all category directories and dummy images are created
for category in categories:
    category_path = os.path.join(data_dir, category)
    if not os.path.exists(category_path):
        os.makedirs(category_path, exist_ok=True)
    # Dummy görüntüler oluştur
    for i in range(5):  # Her kategori için 5 dummy görüntü
        dummy_image_path = os.path.join(category_path, f"dummy_{i}.png")
        if not os.path.exists(dummy_image_path):  # Check if the dummy image already exists
            dummy_image = np.random.randint(0, 256, (img_size, img_size), dtype=np.uint8)
            cv2.imwrite(dummy_image_path, dummy_image)

# Ensure the dummy dataset is created before proceeding
for category in categories:
    category_path = os.path.join(data_dir, category)
    if not os.path.exists(category_path) or len(os.listdir(category_path)) < 5:
        raise FileNotFoundError(f"Dummy dataset directory '{category_path}' was not created successfully or is incomplete.")

# Veri hazırlığı
data = []
labels = []

for category in categories:
    path = os.path.join(data_dir, category)  # Her kategorinin dosya yolu
    class_num = categories.index(category)  # Etiket numarasını al
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            img_array = cv2.resize(img_array, (img_size, img_size))  # Görüntüyü yeniden boyutlandır
            data.append(img_array)
            labels.append(class_num)
        except Exception as e:
            pass

# Verileri numpy dizilerine dönüştür
data = np.array(data).reshape(-1, img_size, img_size, 1)
data = data / 255.0  # Görüntüleri normalize et

labels = to_categorical(labels, num_classes=len(categories))  # Etiketleri one-hot encoding

# Eğitim ve test verisi olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

# Modeli tanımlama
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=X_train.shape[1:]),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(categories), activation='softmax')  # Son katman: her harf için bir çıktı
])

# Modeli derle
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğit
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Modeli kaydet
model.save('Model/keras_model.h5')  # Modeli kaydedin


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3333 - loss: 1.1032 - val_accuracy: 0.3333 - val_loss: 47.5672
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 710ms/step - accuracy: 0.3333 - loss: 44.3521 - val_accuracy: 0.3333 - val_loss: 14.5797
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 699ms/step - accuracy: 0.3333 - loss: 12.1523 - val_accuracy: 0.3333 - val_loss: 18.0575
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 713ms/step - accuracy: 0.3333 - loss: 16.5593 - val_accuracy: 0.3333 - val_loss: 8.7786
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 694ms/step - accuracy: 0.3333 - loss: 7.5856 - val_accuracy: 0.3333 - val_loss: 2.1287
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 673ms/step - accuracy: 0.6667 - loss: 1.5216 - val_accuracy: 0.3333 - val_loss: 1.8525
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 699ms/step - accuracy: 0.6667 - loss: 1.0970 - val_accuracy: 0.3333 - val_loss: 1.8852
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 699ms/step - accuracy: 0.3333 - loss: 0.9051 - val_accuracy: 0.3333 - val_lo